In [3]:
# Cell 1: Imports and setup
import requests
import pandas as pd
from tools import DB

In [4]:
# Cell 2: Constants and helper functions
NUM_PAGES = 100  # Number of TVmaze pages to fetch (each has ~250 shows)

def determine_genre_tag(genres):
    # Use "Anime" if any genre is anime, if not use the first genre or "Unknown" if empty
    if not genres:
        return "Unknown"
    return "Anime" if "anime" in [g.lower() for g in genres] else genres[0]

In [5]:
# Cell 3: DB helper and data structure
db = DB("data.db")
all_shows = {}

In [6]:
# Cell 4: Fetch shows from multiple genres
for page in range(NUM_PAGES):
    print(f"Fetching page {page} of popular shows...")
    response = requests.get(f"https://api.tvmaze.com/shows?page={page}")
    if response.status_code != 200:
        break # Stop if request fails
    for show in response.json():
        show_id = show['id']
        if show_id in all_shows:
            continue # Skip if already added
        genres = show.get('genres', [])
        # Store relevant show info
        all_shows[show_id] = {
            'id': show_id,
            'name': show.get('name'),
            'summary': show.get('summary'),
            'image': show['image']['medium'] if show.get('image') else None,
            'genres': ','.join(genres),
            'genre_tag': determine_genre_tag(genres),
            'showType': show.get('type', 'Unknown'),
            'rating': show['rating']['average'] if show.get('rating') else None,
            'premiered': show.get('premiered'),
            'language': show.get('language'),
            'runtime': show.get('runtime'),
            'country': show['network']['country']['name'] if show.get('network') and show['network'].get('country') else None,
            'network': show['network']['name'] if show.get('network') else None
        }
  

Fetching page 0 of popular shows...
Fetching page 1 of popular shows...
Fetching page 2 of popular shows...
Fetching page 3 of popular shows...
Fetching page 4 of popular shows...
Fetching page 5 of popular shows...
Fetching page 6 of popular shows...
Fetching page 7 of popular shows...
Fetching page 8 of popular shows...
Fetching page 9 of popular shows...
Fetching page 10 of popular shows...
Fetching page 11 of popular shows...
Fetching page 12 of popular shows...
Fetching page 13 of popular shows...
Fetching page 14 of popular shows...
Fetching page 15 of popular shows...
Fetching page 16 of popular shows...
Fetching page 17 of popular shows...
Fetching page 18 of popular shows...
Fetching page 19 of popular shows...
Fetching page 20 of popular shows...
Fetching page 21 of popular shows...
Fetching page 22 of popular shows...
Fetching page 23 of popular shows...
Fetching page 24 of popular shows...
Fetching page 25 of popular shows...
Fetching page 26 of popular shows...
Fetching pa

In [7]:
# Cell 5: Convert to DataFrame and load into the database
df = pd.DataFrame(list(all_shows.values()))
print(df.head())  # Debug: See the first few rows and columns

try:
    db.load_from_dataframe(df, "shows")
    print(f"✅ Loaded {len(df)} shows from TVmaze with showType.")
except Exception as e:
    print("❌ Failed to load DataFrame into database:", e)

   id                name                                            summary  \
0   1      Under the Dome  <p><b>Under the Dome</b> is the story of a sma...   
1   2  Person of Interest  <p>You are being watched. The government has a...   
2   3              Bitten  <p>Based on the critically acclaimed series of...   
3   4               Arrow  <p>After a violent shipwreck, billionaire play...   
4   5      True Detective  <p>Touch darkness and darkness touches you bac...   

                                               image  \
0  https://static.tvmaze.com/uploads/images/mediu...   
1  https://static.tvmaze.com/uploads/images/mediu...   
2  https://static.tvmaze.com/uploads/images/mediu...   
3  https://static.tvmaze.com/uploads/images/mediu...   
4  https://static.tvmaze.com/uploads/images/mediu...   

                           genres genre_tag  showType  rating   premiered  \
0  Drama,Science-Fiction,Thriller     Drama  Scripted     6.5  2013-06-24   
1    Action,Crime,Science-Fi